#### PIPE - 3:    
This pipe covers the following : 
1. Blurring the whole image except a region.
2. Creating evidance folders with object id and all frames relation to the object
3. Creating video clips with tracking object in focus. 

In [1]:
import cv2
import numpy as np
import pandas as pd
import torch
import time
from ultralytics import YOLO
from pathlib import Path
from boxmot import StrongSORT
from utils.utils import *

down = {}
up = {}
text_color = (0, 0, 0)  # Black color for text
yellow_color = (0, 255, 255)  # Yellow color for background
red_color = (0, 0, 255)  # Red color for lines
blue_color = (255, 0, 0)  # Blue color for lines

counter_down = []
counter_up = []

first_boundry_y=170

red_line_y = 198

blue_line_y = 280

offset = 6
# Specify the start and end points of the line
start_point = (300, 198)
end_point = (300, 280)

all_id_tracker=[]
voilation_id_tracker=[]
violation_frame_tracker={}
tracker=tracker_init(cuda_device=torch.cuda.is_available(),cuda_device_number=1)
model=YOLO('yolov8s.pt')
video_path='/media/hlink/hd/test_videos/testx_vid_1.mp4'
vid = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_main = cv2.VideoWriter('output_strongs.avi', fourcc, 20.0, (1020, 500))
frame_count=0
while True:    
    ret, im = vid.read()
    
    if not ret:
        break
    
    im=cv2.resize(im,(1020, 500))
    
    
    results=model.predict(im,conf=0.4,verbose=False,device=[1],classes=[2,3])
    im=line_plotter(frame=im,x_start=300,x_end=774,y=198,line_color=red_color,line_name='boundry 1',line_thickness=2,text_color=text_color)
    im=line_plotter(frame=im,x_start=300,x_end=800,y=280,line_color=blue_color,line_name='boundry 2',line_thickness=2,text_color=text_color)
    
    # Specify the color of the line in BGR format (here, it's white)
    color = (255, 255, 255)
    all_frames_record_path=create_directory('all_frames_record')
    violation_frames_record_path=create_directory('all_violation_record')
    
    # Draw the vertical line on the image
    thickness = 2  # You can adjust the thickness as needed
    cv2.line(im, start_point, end_point, color, thickness)
    
    # im=cv2.rectangle(im, (300, 198), (800, 280), (255, 255, 255), 2)
    
    px,conf=prediction_coordinated_hadler(results)
    dets = []
    # Experimenting
    dets=tracker_element_creator(dets,px,conf)        
    
    # print('out_from')
    
    dets = np.array(dets)
    # print(dets)
    if len(dets) > 0:
        tracks = tracker.update(dets, im) # --> M X (x, y, x, y, id, conf, cls, ind)
        # print('tracks',tracks)    
        for track in tracks:
                im,object_id,cx,cy,bbox_list=plot_tracks(track,im)
               
                all_id_tracker.append(object_id)
                # Adding file path for csv
                csv_file_path=detection_coordinate_write(frame_count,object_id,bbox_list,'all_frame_detection_detail.csv')
            
                if red_line_y<(cy+offset) and red_line_y > (cy-offset):
                    # print("entered if 1")
                    down[object_id]=time.time()
            
                if object_id in down:
                    # print("entered if 2")
               
                    if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
                         # print("entered if 3")
                         elapsed_time=time.time() - down[object_id] 
                         # print("entered if 4")
                         if counter_down.count(object_id)==0:
                            counter_down.append(object_id) 
                            distance = 9
                            est_speed=speed_calculator_kmph(distance,elapsed_time)
                            if est_speed>20:
                                print(f'violation_detected : {object_id}')
                                voilation_id_tracker.append(object_id)
                                violation_frame_tracker[object_id]=frame_count
                                violation_frame_writer_op=frame_writer(frame_count,im,violation_frames_record_path)
                                violation_csv_file_path=detection_coordinate_write(frame_count,object_id,bbox_list,'violation_frame_detection_detail.csv')
                                
                            print(f"for {object_id} speed is -> {est_speed}")
    
    all_frame_writer_op=frame_writer(frame_count,im,all_frames_record_path)    
    frame_count+=1
    out_main.write(im)
    
    # # break on pressing q or spaceq
    cv2.imshow('BoxMOT detection', im)     
    # key = cv2.waitKey(25) & 0xFF
    
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()
print(all_id_tracker)
print(voilation_id_tracker)

2024-05-06 17:04:36.764 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "models/osnet_x0_25_msmt17.pt"


violation_detected : 5
for 5 speed is -> 31
violation_detected : 10
for 10 speed is -> 36
violation_detected : 28
for 28 speed is -> 34
violation_detected : 34
for 34 speed is -> 36
violation_detected : 72
for 72 speed is -> 32
violation_detected : 88
for 88 speed is -> 26
violation_detected : 107
for 107 speed is -> 38
violation_detected : 113
for 113 speed is -> 36
violation_detected : 174
for 174 speed is -> 24
for 192 speed is -> 16
violation_detected : 169
for 169 speed is -> 22
violation_detected : 180
for 180 speed is -> 32
violation_detected : 213
for 213 speed is -> 38
violation_detected : 214
for 214 speed is -> 45
violation_detected : 220
for 220 speed is -> 28
violation_detected : 223
for 223 speed is -> 29
for 232 speed is -> 11
violation_detected : 239
for 239 speed is -> 43
violation_detected : 241
for 241 speed is -> 44
violation_detected : 274
for 274 speed is -> 41
violation_detected : 263
for 263 speed is -> 33
violation_detected : 315
for 315 speed is -> 44
violatio

In [3]:
main_violation_json=voilation_capture_json_creator(voilation_id_tracker,csv_file_path)

In [4]:
for i in main_violation_json:
    order_maintained=frame_order_checker(main_violation_json[i])
    print(order_maintained)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [5]:
evidance_img_dir_paths,evidance_clip_dir_paths=evidance_directories_creator(main_violation_json)

In [6]:
evidance_img_dir_paths

['/home/hlink/workspace/learning/boxmot/06_05_2024_infenrence_output/evidance_images/5',
 '/home/hlink/workspace/learning/boxmot/06_05_2024_infenrence_output/evidance_images/10',
 '/home/hlink/workspace/learning/boxmot/06_05_2024_infenrence_output/evidance_images/28',
 '/home/hlink/workspace/learning/boxmot/06_05_2024_infenrence_output/evidance_images/34',
 '/home/hlink/workspace/learning/boxmot/06_05_2024_infenrence_output/evidance_images/72',
 '/home/hlink/workspace/learning/boxmot/06_05_2024_infenrence_output/evidance_images/88',
 '/home/hlink/workspace/learning/boxmot/06_05_2024_infenrence_output/evidance_images/107',
 '/home/hlink/workspace/learning/boxmot/06_05_2024_infenrence_output/evidance_images/113',
 '/home/hlink/workspace/learning/boxmot/06_05_2024_infenrence_output/evidance_images/174',
 '/home/hlink/workspace/learning/boxmot/06_05_2024_infenrence_output/evidance_images/169',
 '/home/hlink/workspace/learning/boxmot/06_05_2024_infenrence_output/evidance_images/180',
 '/hom

In [7]:
def frame_copier(image_list,source_folder_path,destination_folder_path):
    "Used to copy multiple frame to a destions from a source using frame number I"
    for j in image_list:
            image_name=f'{j[0]}.jpg'
            source_path=os.path.join(source_folder_path,image_name)
            destination_path=os.path.join(destination_folder_path,image_name)
            shutil.copy2(source_path,destination_path)
    
    
def evidance_img_separator(evidance_img_dir_paths,source_folder_path,main_violation_json):
    "Used to copy evidance images into unique folders I"

    for i in zip(main_violation_json,evidance_img_dir_paths):
            temp_arr=main_violation_json[i[0]]
            # print(main_violation_json[i[0]])
            frame_copier(temp_arr,source_folder_path,i[1])
    return evidance_img_dir_paths  

In [8]:
evidance_img_dir_paths=evidance_img_separator(evidance_img_dir_paths,all_frames_record_path,main_violation_json)
video_clip_creator(voilation_id_tracker,violation_frame_tracker,evidance_clip_dir_paths,all_frames_record_path)

In [9]:
# Code for creating video clips for each and every evidance
# Function to sort files by creation time
def sort_files_by_creation_time(folder_path):
    files = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            files.append((file_path, os.path.getctime(file_path)))  # Store file path and creation time

    sorted_files = sorted(files, key=lambda x: x[1])  # Sort files based on creation time
    sorted_file_paths = [file_path for file_path, _ in sorted_files]  # Extract file paths
    return sorted_file_paths
def sort_files_by_name(folder_path):
    files = os.listdir(folder_path)
    sorted_files = sorted(files, key=lambda x: int(x.split('.')[0]))
    file_paths = [os.path.join(folder_path, file) for file in sorted_files]
    return file_paths

In [10]:
evidance_img_dir_paths

['/home/hlink/workspace/learning/boxmot/06_05_2024_infenrence_output/evidance_images/5',
 '/home/hlink/workspace/learning/boxmot/06_05_2024_infenrence_output/evidance_images/10',
 '/home/hlink/workspace/learning/boxmot/06_05_2024_infenrence_output/evidance_images/28',
 '/home/hlink/workspace/learning/boxmot/06_05_2024_infenrence_output/evidance_images/34',
 '/home/hlink/workspace/learning/boxmot/06_05_2024_infenrence_output/evidance_images/72',
 '/home/hlink/workspace/learning/boxmot/06_05_2024_infenrence_output/evidance_images/88',
 '/home/hlink/workspace/learning/boxmot/06_05_2024_infenrence_output/evidance_images/107',
 '/home/hlink/workspace/learning/boxmot/06_05_2024_infenrence_output/evidance_images/113',
 '/home/hlink/workspace/learning/boxmot/06_05_2024_infenrence_output/evidance_images/174',
 '/home/hlink/workspace/learning/boxmot/06_05_2024_infenrence_output/evidance_images/169',
 '/home/hlink/workspace/learning/boxmot/06_05_2024_infenrence_output/evidance_images/180',
 '/hom

In [11]:
def frame_picker(count_id,evidance_folder_path,slider_window_size): 
    all_saved_frames=sort_files_by_creation_time(evidance_folder_path)
    previous_frames=[]
    post_frames=[]
    total_saved_frames=len(all_saved_frames)    
    previous_frame_counter=count_id-slider_window_size
    post_frames_counter=count_id+slider_window_size
    if count_id>slider_window_size and total_saved_frames>slider_window_size:
        for j in range(previous_frame_counter,count_id+1):
                previous_frames.append(all_saved_frames[j])
    else:
        previous_frames=all_saved_frames[:count_id]
    if total_saved_frames>post_frames_counter:
        for i in range(count_id+1,post_frames_counter):
            post_frames.append(all_saved_frames[i])    
    else:
        post_frames=all_saved_frames[count_id:-1]
    return previous_frames,post_frames

In [12]:
violation_frame_tracker

{5: 223,
 10: 254,
 28: 565,
 34: 797,
 72: 1573,
 88: 1720,
 107: 2465,
 113: 2869,
 174: 3694,
 169: 3744,
 180: 3820,
 213: 4260,
 214: 4338,
 220: 4357,
 223: 4563,
 239: 4885,
 241: 4930,
 274: 5673,
 263: 5690,
 315: 6562,
 320: 6775,
 367: 7821,
 401: 8777,
 422: 9110,
 455: 9355,
 522: 10309,
 530: 10539,
 557: 10753,
 637: 11594,
 650: 12065,
 735: 13133,
 742: 13201,
 794: 14004,
 852: 14755}

In [13]:
# temp_arr=violation_frame_tracker[0]

KeyError: 0

In [15]:
for i in voilation_id_tracker:
    print(i)
    
    print(violation_frame_tracker[i])


5
223
10
254
28
565
34
797
72
1573
88
1720
107
2465
113
2869
174
3694
169
3744
180
3820
213
4260
214
4338
220
4357
223
4563
239
4885
241
4930
274
5673
263
5690
315
6562
320
6775
367
7821
401
8777
422
9110
455
9355
522
10309
530
10539
557
10753
637
11594
650
12065
735
13133
742
13201
794
14004
852
14755


In [21]:
def evidance_video_creator(input_file_list,output_file_name='evidance_x',output_folder_path='evidance_clips'):
    temp_path=os.path.join(output_folder_path,f'{output_file_name}.avi')
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(f'{temp_path}', fourcc, 20.0, (1020, 500))
    for i in input_file_list:
        frame = cv2.imread(i)
        out.write(frame)
    out.release()
    return temp_path

In [35]:
violation_frame_tracker[5]

223

In [40]:
def video_clip_creator(voilation_id_tracker,violation_frame_tracker,evidance_clip_dir_paths,all_frames_record_path):
    for i in zip(voilation_id_tracker,violation_frame_tracker,evidance_clip_dir_paths):
        
        previous_frames,post_frames=frame_picker(violation_frame_tracker[i[0]],all_frames_record_path,20)
        input_frame_list=previous_frames+post_frames
        evidance_video_creator(input_frame_list,f'evidance_{i[0]}',output_folder_path=i[2])
    return evidance_clip_dir_paths

5
223
10
254
28
565
34
797
72
1573
88
1720
107
2465
113
2869
174
3694
169
3744
180
3820
213
4260
214
4338
220
4357
223
4563
239
4885
241
4930
274
5673
263
5690
315
6562
320
6775
367
7821
401
8777
422
9110
455
9355
522
10309
530
10539
557
10753
637
11594
650
12065
735
13133
742
13201
794
14004
852
14755
